In [15]:
import sys
sys.path.append('C:\\Users\\user\\PycharmProjects\\pipeGEM')

from pathlib import Path
import cobra
import pandas as pd
import numpy as np
import pipeGEM as pg
from pipeGEM.utils import load_model, save_model, get_subsystems
from pipeGEM.pipeline.algo import SwiftCore, SwiftCC, rFastCormics, FastCC
from pipeGEM.pipeline.threshold import BimodalThreshold


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
model_path = "../models/2022Feb/"

In [3]:
mg_TPMs = pd.read_csv("../data/mg_TPMs.tsv", sep='\t')

mg_TPMs["gene"] = mg_TPMs["gene"].apply(lambda x: x[:x.index(".")] if "." in x else x)
mg_TPMs = mg_TPMs.groupby("gene").sum()

In [24]:
human = load_model("../models/Human-GEM.mat")
# human = load_model("../models/chuman.mat")
for r in human.reactions:
    r.subsystem = r.subsystem[r.subsystem.index("(['")+3: r.subsystem.index("']")]

Academic license - for non-commercial use only


In [21]:
dmem_medium_path = Path("../data/ac16_fluxes.csv")
dmem_medium = pd.read_csv(dmem_medium_path, index_col=0)
rxn_map = {}
ex_rxns = [r.id for r in human.exchanges]
for m in human.metabolites:
    if not any([(met+ "s")== (m.id) for met in dmem_medium["human_gem"] if pd.notna(met)]) :
        continue
    for r in m.reactions:
        if r.id in ex_rxns:
            rxn_map[m.id[:-1]] = r.id
dmem_medium["human_gem_rxn"] = dmem_medium["human_gem"].map(rxn_map)

In [29]:
inorg_met_ids = ['MAM02200','MAM02609', 'MAM02949','MAM02946',
                 'MAM01821','MAM01822', 'MAM01442','MAM01413',
                 'MAM03157','MAM02751', 'MAM02147','MAM02040',
                 'MAM02588','MAM02630', 'MAM02759','MAM02579',
                 'MAM03157','MAM01596', 'MAM02959','MAM02519', 'MAM02046']
rxn_map = {}
ex_rxns = [r.id for r in human.exchanges]
inorg_rxns = []
for m in human.metabolites:
    if not any([(met+ "s")== (m.id) for met in inorg_met_ids if pd.notna(met)]) :
        continue
    for r in m.reactions:
        if r.id in ex_rxns:
            rxn_map[m.id[:-1]] = r.id
            inorg_rxns.append(r.id)

In [32]:
for r in human.exchanges:
    if r.id in inorg_rxns:
        r.lower_bound = -100
    elif r.id in dmem_medium["human_gem_rxn"]:
        r.lower_bound = -10
    else:
        r.lower_bound = -0.01

In [44]:
c = FastCC()
c(human, tol=1e-4, is_convex=False)

c_model = c.output["model"].copy()
save_model(c_model, "../models/c_human", ".json")

Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpexvzc2ea.lp
Reading time = 0.06 seconds
: 7126 rows, 23768 columns, 99148 nonzeros


In [42]:
rfc = rFastCormics(consist_method = None,
                   task_file_path = "../data/general_task_human_gem_s.json",
                   task_constr_name = "None",
                   model_compartment_format = "{}",
                   solver="gurobi",
                   cc_threshold=1e-4,
                   core_threshold=1e-4,
                   use_first_guess=False)


output = rfc(c_model,
             mg_TPMs,
             protected_rxns=["MAR13082"],
             not_penalized_subsystem=r"Transport reactions")

for k, v in output['final_model_dic'].items():
    save_model(v, model_path + k, ".json")